ДЕМОНСТРАЦИЯ РЕШЕНИЯ. 
ДЛЯ ТОГО ЧТОБЫ РЕШЕНИЕ РАБОТАЛО НУЖНО УСТАНОВИТЬ ЗАВИСИМОСТИ ИЗ ФАЙЛА В РЕПОЗИТОРИИ + СКАЧАТЬ ПРЕДОБУЧЕННЫЕ МОДЕЛИ

In [24]:
#импорты нужных библиотек
import cv2
import yaml
from ultralytics import YOLO
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import os

In [25]:
import cv2
import yaml
from ultralytics import YOLO
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import math

class VideoProcessor:
    '''
    Класс предназначен для обработки 1 видео на предмет нарушений ПДД.
    Класс работает с изображением в цветовом пространстве RGB.
    '''
    def __init__(self, sign_model_path, traffic_light_model_url, sign_classes_yaml):
        """
        Инициализация класса для обработки видео с использованием моделей для распознавания знаков и светофоров.
        
        :param sign_model_path: Путь к файлу модели для распознавания знаков (YOLO).
        :param traffic_light_model_url: Ссылка на модель для распознавания светофоров.
        :param sign_classes_yaml: Путь к файлу в формате YAML, который содержит классы знаков.
        :param lane_detection_algo_path: Путь к алгоритму для распознавания разметки (пока не реализовано).
        """
        self.sign_model = YOLO(sign_model_path)
        self.traffic_light_model = YOLO(traffic_light_model_url)
        self.traffic_signs = {"2.5": "Движение без остановки запрещено", "1.12": "Разметка стоп-линия", "5.15.1": "Полоса для маршрутных транспортных средств", "5.15.2": "Полоса для маршрутных транспортных средств", "3.1": "Въезд запрещен (кирпич)", "1.4.1": "Обозначение полос движения", "4.1.1": "Движение прямо", "2.1": "Главная дорога", "3.27": "Остановка запрещена", "3.28": "Стоянка запрещена", "3.18.1": "Поворот налево запрещен", "3.18.2": "Разворот запрещен", "4.1.3": "Движение налево", "3.20": "Обгон запрещен", "1.1": "Сплошная линия разметки"}

        # Загружаем классы знаков из YAML
        with open(sign_classes_yaml, 'r') as f:
            yaml_dict = yaml.safe_load(f)
        self.sign_classes = yaml_dict['names']

    def draw_annotations(self, frame, results:dict):
        """
        Рисует аннотации на изображении на основе результатов распознавания объектов.

        :param frame: Кадр изображения, на котором будут отображаться аннотации.
        :param results: Словарь с результатами распознавания, где ключи — метки объектов, 
                        а значения — списки координат их bounding boxes.
                        Формат словаря: {'label1': [(x1, y1, w1, h1), ...], 'label2': [(x2, y2, w2, h2), ...], ...}
        
        Описание:
        - Функция проходит по всем меткам и их bounding boxes в словаре результатов и рисует их на изображении.
        - Рисует прямоугольники вокруг обнаруженных объектов и отображает метки классов рядом с ними.
        """
        for label, bboxes in results.items():
            for x, y, w, h in bboxes:
                cv2.rectangle(frame, (int(x - w / 2), int(y - h / 2)),
                            (int(x + w / 2), int(y + h / 2)), (0, 255, 0), 2)
                # Отображаем метку класса рядом с bbox
                cv2.putText(frame, label, (int(x - w / 2), int(y - h / 2) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                plt.imshow(frame)
                plt.show()

    def process_traffic_signs(self, frame, confidence=0.2):
            """
            Функция для обработки кадра и распознавания дорожных знаков.
            
            :param confidence: уверенность модели в правильной идентификации объекта.
            :param frame: Входной кадр для обработки.
            :return: Список с типами знаков и их координатами.
            """
            results = self.sign_model(frame)
            boxes = results[0].boxes.xywh.tolist()
            cls = results[0].boxes.cls.tolist()
            conf = results[0].boxes.conf
            sign_bboxes = []

            # Проходим по всем найденным объектам и определяем их тип
            for (box, cls_, conf_) in zip(boxes, cls, conf):  # Перебор объектов

                x, y, w, h = box
                class_name = self.sign_classes[int(cls_)]
                if conf_ > confidence:
                
                    sign_bboxes.append((class_name, (x, y, w, h)))
            
            return sign_bboxes
    
    def crop_and_identify_color(self, frame, bbox):
        """
        Обрезает изображение по координатам bounding box и определяет цвет светофора.

        :param frame: Исходное изображение (кадр) для обработки.
        :param bbox: Координаты bounding box в формате (x, y, w, h), где x и y - координаты центра, 
                    w и h - ширина и высота.
        :return: Целочисленное значение, обозначающее преобладающий цвет:
                1 - красный или желтый (сигналы, означающие остановку),
                0 - зеленый (сигнал продолжения движения),
                None - цвет не определен.
        """
        # bbox — координаты в формате (x, y, w, h)
        x, y, w, h = bbox

        # Определение верхнего левого и нижнего правого углов
        x1 = int(x - w / 2)
        y1 = int(y - h / 2)
        x2 = int(x + w / 2)
        y2 = int(y + h / 2)

        # Обрезка изображения по заданным координатам
        cropped_image = frame[y1:y2, x1:x2]

        # Преобразование в цветовое пространство HSV
        hsv_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_RGB2HSV)

        # Определение диапазонов цвета для красного, желтого и зеленого
        red_lower1 = np.array([0, 70, 50])
        red_upper1 = np.array([10, 255, 255])
        red_lower2 = np.array([170, 70, 50])
        red_upper2 = np.array([180, 255, 255])
        green_lower = np.array([35, 70, 50])
        green_upper = np.array([85, 255, 255])
        yellow_lower = np.array([20, 70, 50])
        yellow_upper = np.array([30, 255, 255])

        # Маски для определения цветов
        mask_red = cv2.inRange(hsv_cropped, red_lower1, red_upper1) + cv2.inRange(hsv_cropped, red_lower2, red_upper2)
        mask_green = cv2.inRange(hsv_cropped, green_lower, green_upper)
        mask_yellow = cv2.inRange(hsv_cropped, yellow_lower, yellow_upper)

        # Подсчет количества пикселей каждого цвета
        red_count = np.sum(mask_red > 0)
        green_count = np.sum(mask_green > 0)
        yellow_count = np.sum(mask_yellow > 0)

        # Определение преобладающего цвета
        if red_count > green_count and red_count > yellow_count:
            return 1
        elif green_count > red_count and green_count > yellow_count:
            return 0
        elif yellow_count > red_count and yellow_count > green_count:
            return 1
        else:
            return 0
        
    def process_traffic_lights_and_stop_signs(self, frame, confidence=0.25):
        """
        Функция для обработки кадра и распознавания светофоров и стоп-знаков.
        
        :param frame: Входной кадр для обработки.
        :param confidence: уверенность модели в правильной идентификации объекта.
        :return: Список с координатами для светофоров и стоп-знаков.
        """
        results = self.traffic_light_model(frame)
        boxes = results[0].boxes.xywh.tolist()
        cls = results[0].boxes.cls.tolist()
        conf = results[0].boxes.conf
        bboxes_traffic_lights = []
        bboxes_stop_signs = []
        traffic_lights_colors = []

        # Проходим по всем найденным объектам и определяем их тип
        for (box, cls_, conf_) in zip(boxes, cls, conf):  # Перебор объектов

            x, y, w, h = box
            class_name = results[0].names[int(cls_)]
            if conf_ > confidence:
                if class_name.lower() == 'traffic light':  # Если это светофор
                    bboxes_traffic_lights.append((x, y, w, h))
                    traffic_lights_colors.append(self.crop_and_identify_color(frame, box))
        
        return bboxes_traffic_lights, traffic_lights_colors

    
    #def process_frame(self, frame):
        """
        Обработка одного кадра: распознавание светофоров, стоп-знаков и дорожных знаков.
        
        :param frame: Входной кадр для обработки.
        :return: Словарь с результатами по каждому типу объектов.
        """
        # Получаем координаты светофоров и стоп-знаков
        bboxes_traffic_lights, color = self.process_traffic_lights_and_stop_signs(frame)
        
        # Получаем координаты дорожных знаков
        sign_bboxes = self.process_traffic_signs(frame)
        
        return {
            'traffic_lights': bboxes_traffic_lights,
            'traffic_signs': sign_bboxes,
            'color': color
        }

    def draw_annotations(self, frame, results:dict):
        """
        Рисует аннотации на изображении на основе результатов распознавания объектов.

        :param frame: Кадр изображения, на котором будут отображаться аннотации.
        :param results: Словарь с результатами распознавания, где ключи — метки объектов, 
                        а значения — списки координат их bounding boxes.
                        Формат словаря: {'label1': [(x1, y1, w1, h1), ...], 'label2': [(x2, y2, w2, h2), ...], ...}
        
        Описание:
        - Функция проходит по всем меткам и их bounding boxes в словаре результатов и рисует их на изображении.
        - Рисует прямоугольники вокруг обнаруженных объектов и отображает метки классов рядом с ними.
        """
        for label, bboxes in results.items():
            for x, y, w, h in bboxes:
                cv2.rectangle(frame, (int(x - w / 2), int(y - h / 2)),
                            (int(x + w / 2), int(y + h / 2)), (0, 255, 0), 2)
                # Отображаем метку класса рядом с bbox
                cv2.putText(frame, label, (int(x - w / 2), int(y - h / 2) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    def img_detect_color(self, frame, show=False):
        """
        Функция для выделения светлых (приблизительно белых) областей на изображении.

        Параметры:
        image (numpy.ndarray): Входное изображение.
        show (bool): Если True, отображает результат обработки.

        Возвращает:
        numpy.ndarray: Изображение, на котором выделены светлые области.
        """

        # Создаем копию изображения для обработки
        color_select = np.copy(frame)

        # Задаем пороговые значения для цветовых каналов (R, G, B)
        red_threshold = 130
        green_threshold = 130
        blue_threshold = 120

        # Создаем маску для пикселей, не соответствующих пороговым значениям
        # Пиксели, значения цветовых каналов которых ниже пороговых значений, будут отфильтрованы
        thresholds = (frame[:, :, 0] < red_threshold) | \
                    (frame[:, :, 1] < green_threshold) | \
                    (frame[:, :, 2] < blue_threshold)

        # Применяем маску: пиксели, не соответствующие пороговым значениям, окрашиваются в черный цвет
        color_select[thresholds] = [0, 0, 0]

        # Отображаем результат, если параметр show установлен в True
        if show:
            plt.imshow(color_select)
            plt.title("Выделение +- белого цвета")
            plt.show()

        # Возвращаем изображение с выделенными светлыми областями
        return color_select


    def mask_area_on_image(self, image, show=False):
        """
        Функция для выделения области дороги на изображении с наложением маски.

        Параметры:
        image (numpy.ndarray): Входное изображение.
        show (bool): Если True, отображает результат обработки и границу области интереса.

        Возвращает:
        numpy.ndarray: Изображение с выделенной областью.
        """

        # Создаем пустую маску того же размера, что и изображение (трехканальную для цветного изображения)
        mask = np.zeros_like(image)
        height, width, _ = mask.shape  # Получаем высоту, ширину и количество каналов изображения

        # Определяем координаты полигона для выделения области интереса (в данном случае - дороги перед машиной)
        # polygon - массив с координатами углового полигона (ROI)
        polygon = np.array([[
        (int(width * 0.4), height),           # Левый нижний угол
        (int(width * 0.6), height),           # Правый нижний угол
        (int(width * 0.6), int(height * 0.7)), # Правый верхний угол
        (int(width * 0.4), int(height * 0.7)), # Левый верхний угол
        (int(width * 0.4), height - (0.2 * height)), # Левый нижний угол
        (int(width * 0.5), height - (0.2 * height)), # Правый нижний угол
        (int(width * 0.5), int(height * 0.6)), # Правый верхний угол
        (int(width * 0.4), int(height * 0.6))  # Левый верхний угол
        ]], np.int32)



        # Заполняем маску полигоном белого цвета (255, 255, 255) для трехканального изображения
        #cv2.fillPoly(mask, polygon, (255, 255, 255))
        cv2.fillPoly(mask, [polygon], (255, 255, 255))

        # Применяем маску к изображению с помощью побитового И, оставляя только область в форме полигона
        masked_image = cv2.bitwise_and(image, mask)

        # Отображаем результат, если параметр show установлен в True
        if show:
            # Создаем копию изображения с наложенной маской для добавления границы
            image_with_border = masked_image.copy()

            # Наносим красный контур на границы полигона
            cv2.polylines(image_with_border, [polygon], isClosed=True, color=(255, 0, 0), thickness=1)

            # Отображаем изображение с наложенной маской и красной границей
            plt.imshow(image_with_border)
            plt.title("Выделение региона дороги на изображении")
            plt.show()

        # Возвращаем изображение, на которое наложена маска
        return masked_image




    def lines_detect(self, image, show=False):
        """
        Функция для обнаружения линий на изображении с использованием Canny edge detection.

        Параметры:
        image (numpy.ndarray): Входное изображение.
        show (bool): Если True, отображает результат обработки.

        Возвращает:
        numpy.ndarray: Изображение с выделенными границами.
        """

        # Преобразуем изображение в оттенки серого, так как алгоритм Canny работает с монохромными изображениями
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        # Применяем размытие для уменьшения шума на изображении и сглаживания переходов
        # Используется фильтр Гаусса с ядром 5x5
        blur = cv2.GaussianBlur(gray, (5, 5), 0)

        # Применяем оператор Canny для выделения границ
        # Первый и второй параметры - пороговые значения для градиента (нижний и верхний)
        edges = cv2.Canny(blur, 30, 100)  # 30 и 100 - пороговые значения

        # Отображаем результат, если параметр show установлен в True
        if show:
            # Отображаем изображение с выделенными линиями
            plt.imshow(edges, cmap='gray')
            plt.title("Выделение линий")
            plt.show()

        # Возвращаем изображение с выделенными границами
        return edges


    def detect_road_marking(self, base_image, image, show=False):
        """
        Функция для обнаружения дорожной разметки на изображении с использованием Hough Line Transform.

        Параметры:
        base_image (numpy.ndarray): Исходное изображение, на которое будут нанесены обнаруженные линии.
        image (numpy.ndarray): Обработанное изображение (например, после edge detection), на котором ищем линии.
        show (bool): Если True, отображает результат обработки.

        Возвращает:
        list: Список обнаруженных линий (или пустой список, если линии не обнаружены).
        """

        # Задаем параметры Hough Line Transform
        threshold_value = 65  # Пороговое значение для минимального количества пересечений для обнаружения линии
        min_line_length = 60  # Минимальная длина линии, ниже которой линия не будет детектироваться
        max_line_gap = 50  # Максимальное расстояние между отрезками, при котором они считаются одной линией

        # Выполняем поиск линий с использованием метода HoughLinesP
        lines = cv2.HoughLinesP(image, rho=1, theta=np.pi / 180,
                                threshold=threshold_value,
                                minLineLength=min_line_length,
                                maxLineGap=max_line_gap)

        # Отображаем результат, если параметр show установлен в True
        if show:
            # Создаем пустое изображение для отрисовки обнаруженных линий
            line_image = np.zeros_like(base_image)

            # Проверяем, есть ли найденные линии
            if lines is not None:
                for line in lines:
                    print('линия',line)
                    x1, y1, x2, y2 = line[0]  # Извлекаем координаты начала и конца линии
                    cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 5)  # Рисуем линию зеленым цветом

            # Накладываем изображение с линиями на исходный кадр для отображения результатов
            combined_image = cv2.addWeighted(base_image, 0.8, line_image, 1, 0)

            # Отображаем изображение с выделенной дорожной разметкой
            plt.imshow(combined_image)
            plt.title("Выделение дорожной разметки")
            plt.show()

        # Возвращаем найденные линии или пустой список, если линии не обнаружены
        if lines is not None:
            return lines
        else:
            return []

    # Функция для вычисления длины линии
    def line_length(self, x1, y1, x2, y2):
        return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    def does_line_intersect_zone(self, x1, y1, x2, y2, zone_start, zone_end, height):
        """
        Функция для проверки, пересекает ли линия заданную зону на изображении.

        Параметры:
        x1, y1 (int): Координаты начала линии.
        x2, y2 (int): Координаты конца линии.
        zone_start (int): Начальная граница зоны по оси X.
        zone_end (int): Конечная граница зоны по оси X.
        height (int): Высота изображения (для проверки вертикальных границ зоны).

        Возвращает:
        bool: True, если линия пересекает зону; иначе False.
        """

        # Проверяем, попадает ли хотя бы одна из конечных точек линии в зону
        # Если x-координата начала или конца линии находится между zone_start и zone_end, то линия пересекает зону
        #print('начало и конец линии',x1,x2)
        if (zone_start <= x1 <= zone_end) or (zone_start <= x2 <= zone_end):
            return True

        # Если конечные точки линии не попали в зону, проверяем пересечение линии с границами зоны
        # Вычисляем коэффициенты уравнения линии Ax + By + C = 0
        A = y2 - y1  # Разница координат Y
        B = x1 - x2  # Разница координат X
        C = (x2 - x1) * y1 - (y2 - y1) * x1  # Свободный член уравнения

        # Рассчитываем точки пересечения линии с вертикальными границами зоны
        # Если линия пересекает зону на уровне zone_start
        y_start = (-A * zone_start - C) / B if B != 0 else None  # Координата Y пересечения с границей zone_start

        # Если линия пересекает зону на уровне zone_end
        y_end = (-A * zone_end - C) / B if B != 0 else None  # Координата Y пересечения с границей zone_end

        # Проверяем, находится ли точка пересечения в пределах изображения по вертикали (от 0 до height)
        if y_start is not None and (0 <= y_start <= height):
            return True
        if y_end is not None and (0 <= y_end <= height):
            return True

        # Если ни одна из проверок не вернула True, линия не пересекает зону
        return False


    def does_center_intersect_line_center(self, x1, y1, x2, y2, bbox_center_x, bbox_center_y):
        """
        Функция для проверки, пересекает ли центр изображения центральную часть линии.

        Параметры:
        x1, y1 (int): Координаты начала линии.
        x2, y2 (int): Координаты конца линии.
        image_center_x (int): X-координата центра изображения.

        Возвращает:
        bool: True, если центр изображения пересекает центральную часть линии; иначе False.
        """

        # Находим среднюю точку линии
        mid_x = (x1 + x2) / 2  # X-координата средней точки
        mid_y = (y1 + y2) / 2  # Y-координата средней точки (не используется в дальнейшем)

        # Вычисляем длину линии
        line_len = self.line_length(x1, y1, x2, y2)  # Вызываем функцию для вычисления длины линии
        #print('длина линии',line_len)
        offset = abs(0.5*math.cos(math.radians(abs(np.degrees(np.arctan2(abs(y2 - y1), abs(x2 - x1))))))) * line_len  # 20% от длины линии (10% в каждую сторону от средней точки)
        print('угол',abs(np.degrees(np.arctan2(abs(y2 - y1), abs(x2 - x1)))))
        print('offset',offset)
        # Определяем границы центральной части линии по оси X
        center_start_x = mid_x - offset  # Левая граница центральной части линии
        center_end_x = mid_x + offset    # Правая граница центральной части линии

        # Проверяем, пересекает ли центр изображения эту центральную часть линии по оси X
        if center_start_x <= bbox_center_x <= center_end_x:
            return True  # Возвращает True или False


    def line_crossing_check(self, lines, image, min_len_line=100, ignore_horizontal=True, verbose=False):
        """
        Функция для проверки, пересекает ли обнаруженные линии центральную часть изображения.

        Параметры:
        lines (list): Список обнаруженных линий.
        image (numpy.ndarray): Изображение, на котором проводим проверку.
        min_len_line (int): Минимальная длина линии для рассмотрения.
        ignore_horizontal (bool): Если True, игнорирует почти горизонтальные линии.
        verbose (bool): Если True, выводит дополнительную информацию о пересекающих линиях.

        Возвращает:
        bool: True, если найдена пересекающая линия; иначе False.
        """

        # Получаем высоту и ширину изображения
        height, width, _ = image.shape

        # Определяем границы зоны 10% по ширине
        #zone_width = width * 0.1  # Ширина зоны в 10% от ширины изображения
        #zone_start = (width / 2) - (zone_width / 2)  # Левая граница зоны
        #zone_end = (width / 2) + (zone_width / 2)    # Правая граница зоны
        polygon = np.array([[
        (int(width * 0.4), height),           # Левый нижний угол
        (int(width * 0.6), height),           # Правый нижний угол
        (int(width * 0.6), int(height * 0.7)), # Правый верхний угол
        (int(width * 0.4), int(height * 0.7)), # Левый верхний угол
        (int(width * 0.4), height - (0.2 * height)), # Левый нижний угол
        (int(width * 0.5), height - (0.2 * height)), # Правый нижний угол
        (int(width * 0.5), int(height * 0.6)), # Правый верхний угол
        (int(width * 0.4), int(height * 0.6))  # Левый верхний угол
        ]], np.int32)

        bbox_center_x = (polygon[0][0][0] + polygon[0][2][0]) / 2  # X-координата центра
        bbox_center_y = (polygon[0][0][1] + polygon[0][2][1]) / 2  # Y-координата центра

        print ('центр', bbox_center_x, bbox_center_y)
        zone_start = polygon[0][0]  # Левая граница зоны
        zone_end = polygon[2][0]
        #print(zone_start,zone_end)

        # Если список линий пустой, возвращаем False
        if len(lines) == 0:
            print('линии не найдены')
            return False


        # Проверка на количество линий, чтобы избежать шума
        #if len(lines) > 20:
            #return False  # Если слишком много линий, возвращаем False

        # Обрабатываем каждую линию
        for line in lines:

            x1, y1, x2, y2 = line[0]  # Извлечение координат начала и конца линии
            line_len = self.line_length(x1, y1, x2, y2)  # Вычисление длины линии
            print('концы',x1,x2)

            # Проверяем, пересекает ли линия зону и центральную часть изображения
            intersects_zone = self.does_line_intersect_zone(x1, y1, x2, y2, zone_start, zone_end, height)
            center_intersects_line_center = self.does_center_intersect_line_center(x1, y1, x2, y2, bbox_center_x, bbox_center_y)
            print('int',intersects_zone)
            print('cent',center_intersects_line_center)

            # Определяем, какая из точек является левой нижней
            if (y1 > y2) or (y1 == y2 and x1 < x2):
                left_x, left_y = x1, y1  # Если первая точка ниже, она считается левой нижней
            else:
                left_x, left_y = x2, y2  # Иначе считается, что второй точка - левая нижняя

            if line_len< min_len_line:
                print(line_len,'короткая')
                continue

            #if ignore_horizontal:
            #    if abs(y2 - y1) < height * 0.2:  # Если линия почти горизонтальная
            #        continue  # Переходим к следующей линии

            # Проверка наклона линии (игнорируем горизонтальные линии, если указано)
            if ignore_horizontal:
                if not 20<abs(np.degrees(np.arctan2(abs(y2 - y1), abs(x2 - x1))))<160:  # Если линия почти горизонтальная
                    print(abs(np.degrees(np.arctan2(abs(y2 - y1), abs(x2 - x1)))),'горизонтальная')
                    continue  # Переходим к следующей линии

            if intersects_zone and center_intersects_line_center:
                print('cocoint')
                if verbose:
                    # Выводим информацию о пересекающей линии, если verbose=True
                    print(f"Line with length {int(line_len)} intersects the 10% center zone, "
                        f"the lower-left point is right of the center, and "
                        f"the 20% center section of the line intersects the image center.")
                return True  # Если условия выполнены, возвращаем True

            # Условие для проверки длины, пересечения зоны и центральной части линии
            #if center_intersects_line_center:
            #    print('coco')
            #    if verbose:
            #        # Выводим информацию о пересекающей линии, если verbose=True
            #        print(f"Line with length {int(line_len)} intersects the 10% center zone, "
            #              f"the lower-left point is right of the center, and "
            #              f"the 20% center section of the line intersects the image center.")
            #    return True  # Если условия выполнены, возвращаем True


            return False  # Если не нашли подходящих линий, возвращаем False


    def process_frame(self, frame, show = False):
        image = self.img_detect_color(frame, show)
        image = self.mask_area_on_image(image, show)
        image = self.lines_detect(image, show)
        lines = self.detect_road_marking(frame, image, show)
        violation = self.line_crossing_check(lines, frame, min_len_line = 60)

        print('violation',violation)
        return violation


    def process_video(self, video_path, frames_to_take=50, show=False, debug_sec=[]):
        """
        Функция для анализа кадров из видео на предмет нарушений правил.

        Параметры:
        video_path (str): Путь к видеофайлу.
        frames_to_take (int): Количество кадров для анализа.
        show (bool): Если True, отображает текущий кадр.
        debug_sec (list): Список секунд, на которых нужно показать кадры и вывести отладочную информацию.

        Возвращает:
        pd.DataFrame: DataFrame с результатами анализа.
        """

        # Создаем DataFrame для хранения результатов
        columns = ['номер видео', 'наименование нарушения', 'сумма штрафа, руб.', 'время нарушения (в секундах)']
        df_results = pd.DataFrame(columns=columns)

        # Открываем видеофайл
        cap = cv2.VideoCapture(video_path)

        # Получаем параметры видео
        fps = cap.get(cv2.CAP_PROP_FPS)  # Частота кадров
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Ширина кадра
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Высота кадра
        count_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Общее количество кадров
        duration = count_frame // fps  # Длительность видео в секундах
        freq = max(1, count_frame // frames_to_take)  # Частота выборки кадров (не менее 1)
        print(f"Частота выборки кадров: {freq}")

        success, frame = cap.read()  # Читаем первый кадр
        count = 0  # Счетчик кадров

        # Инициализируем tqdm для отслеживания прогресса
        with tqdm(total=count_frame // freq, desc="Processing frames") as pbar:
            while success:
                if count % freq == 0:  # Обрабатываем только выбранные кадры
                    time_sec = count // fps  # Вычисляем текущую секунду видео
                    if time_sec in debug_sec:
                        show = True  # Включаем отображение, если текущая секунда в списке отладки

                    # Обрабатываем кадр (преобразуем цветовую схему)
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    if show:
                        plt.imshow(frame)  # Отображаем текущий кадр
                        plt.title("Current Frame")
                        plt.show()

                    # Вызываем функцию для обработки кадра
                    violation = self.process_frame(frame, show=show)

                    if violation:
                        # Добавляем результат анализа в DataFrame
                        violation_row = {
                            'номер видео': video_path,
                            'наименование нарушения': 'Статья 12.16. часть 1 Несоблюдение требований, предписанных дорожными знаками или разметкой проезжей части дороги',
                            'сумма штрафа, руб.': 500,  # Предполагаемая сумма штрафа
                            'время нарушения (в секундах)': time_sec
                        }
                        df_results = df_results.append(violation_row, ignore_index=True)

                    # Обновляем прогресс-бар
                    pbar.update(1)

                    if time_sec in debug_sec:
                        show = False  # Отключаем отображение после вывода отладки
                        print(f'\n\nОбработка кадра на {time_sec} секунде')
                        print('Правило нарушено' if violation else 'Правило не нарушено')
                        print('----------------------------')

                success, frame = cap.read()  # Читаем следующий кадр
                count += 1  # Увеличиваем счетчик

        cap.release()  # Закрываем видеофайл
        cv2.destroyAllWindows()  # Закрываем все окна OpenCV

        return df_results  # Возвращаем DataFrame с результатами анализа


In [26]:
proc = VideoProcessor('D:\\traffic_violation_checker\\models\\best.pt', 'D:\\traffic_violation_checker\\models\\yolov8s.pt', 'D:\\traffic_violation_checker\\models\\class_labels.yaml')

In [27]:
columns = ['номер видео', 'наименование нарушения', 'сумма штрафа, руб.', 'время нарушения (в секундах)']
sub = pd.DataFrame(columns=columns)

In [28]:
video_folder = 'D:\\train_rzd\\test РЖД ПДД\\videos\\videos' #путь к папке с видео

In [ ]:
# Проход по всем видеофайлам в папке
for video_file in os.listdir(video_folder):
    if video_file.endswith(('.mp4', '.avi', '.mov')):  # Проверка на допустимые форматы видео
        video_path = os.path.join(video_folder, video_file)
        
        # Обработка видео и получение DataFrame с нарушениями
        violations_df = proc.process_video(video_path, frames_to_take=1000, show=False)
        
        # Объединение с основной таблицей sub
        sub = pd.concat([sub, violations_df], ignore_index=True)

In [ ]:
output_path = 'path/to/output_folder/sub_table.csv' #путь к папке с submission.csv
sub.to_csv(output_path, index=False)
print(f"Таблица sub успешно создана и сохранена в {output_path}")